In [2]:
import seaborn as sb
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt 
import scipy.stats as st
from scipy.stats import norm
import math 
sb.set()

import matplotlib as mpl
sb.set(rc = {'figure.figsize':(15,8)})
mpl.rcParams['figure.dpi'] = 300

In [3]:
#generates outcomes for dice and coins given a distribution

def dice(x): 
    if (x<=1/6):
        return 1
    elif (x>1/6 and x<=2/6):
        return 2
    elif (x>2/6 and x<=3/6):
        return 3
    elif (x>3/6 and x<=4/6):
        return 4
    elif (x>4/6 and x<=5/6):
        return 5
    elif (x>5/6 and x<1):
        return 6
    
def coin(x):
    if (x>0 and x<=1/2):
        return 0
    elif (x > 1/2):
        return 1

#generates loss for portfolio PD

def PDLoss(x):
    if x<70:
        return (-700000+PD_0)
    elif (x>=70 and x<=76):
        return (-10000*x+PD_0)
    elif x>76:
        return (-760000+PD_0)




### 1. Simulation of idiosyncratic and systematic factors and estimation
#### Generating Uniform Distributions and Outcomes via Inverse Method

### 1a) 

In [4]:
#creating temporary dataframes
temp=pd.DataFrame()
Exp_T=[]
Exp_T_tilda=[]
a=[]
a_tilda=[]
#this loop resamples the distributions, finds the expected value and quantile given a confidence level
for n in range(1,51):
    temp['X1 Dist']= np.random.rand(1000) 
    temp['X2 Dist']= np.random.rand(1000)
    temp['C1 Dist']= np.random.rand(1000)
    temp['C2 Dist']= np.random.rand(1000)
    temp['X1']=temp['X1 Dist'].apply(lambda x: dice(x))
    temp['X2']=temp['X2 Dist'].apply(lambda x: dice(x))
    temp['C1']=temp['C1 Dist'].apply(lambda x: coin(x))
    temp['C2']=temp['C2 Dist'].apply(lambda x: coin(x))
    temp['T']=temp['X1']+temp['X2']+4*temp['C1']+4*temp['C2']
    temp['T_tilda']=temp['X1']+temp['X2']+8*temp['C1']
    
    Exp_T.append(temp.query("T >= 16")["T"].mean())
    Exp_T_tilda.append(temp.query("T_tilda >= 16")["T_tilda"].mean())
    a.append(temp.sort_values(by=['T']).iloc[970]['T'])
    a_tilda.append(temp.sort_values(by=['T_tilda']).iloc[970]['T_tilda'])
    
#this function, then estimates the 95% CI for the list of probabilities
CI_T = st.norm.interval(alpha=0.95, loc=np.mean(Exp_T), scale=st.sem(Exp_T))
CI_T_Tilda = st.norm.interval(alpha=0.95, loc=np.mean(Exp_T_tilda), scale=st.sem(Exp_T_tilda))
CI_a = st.norm.interval(alpha=0.95, loc=np.mean(a), scale=st.sem(a))
CI_a_tilda = st.norm.interval(alpha=0.95, loc=np.mean(a_tilda), scale=st.sem(a_tilda))

In [5]:
#creates dataframe to present solutions neatly
Frame=pd.DataFrame(index=['T', 'T Tilda', 'a', 'a Tilda'])
Frame['Estimate'] = [np.mean(Exp_T), np.mean(Exp_T_tilda), np.mean(a), np.mean(a_tilda)]
Frame['Confidence Interval'] = [CI_T, CI_T_Tilda, CI_a, CI_a_tilda]
Frame['Confidence Interval'] = Frame['Confidence Interval'].apply(lambda x: np.round(x,3))
Frame.round(3)

,Estimate,Confidence Interval
T,17.178,"[17.147, 17.21]"
T Tilda,17.338,"[17.31, 17.366]"
a,17.960,"[17.905, 18.015]"
a Tilda,18.960,"[18.905, 19.015]"


### 1b)

All the estimates are very close to their true values and also lie within the range of their respective and masterfully crafted confidence intervals. Similarly, there is no confidence interval for a tilda, because all of its values the same.

### 2. Evaluating Hedge Options Strategies 

#### 2d)

In [6]:
#generating simulated stock prices 
Sim=pd.DataFrame()
S_0 = 67.45
mu=0.045
sig=0.2
T=11/52
Sim['U']=np.random.rand(5000)
Sim['Z']=Sim['U'].apply(lambda x:norm.ppf(x))
Sim['X_T']= (mu-(sig**2)/2)*T + sig*np.sqrt(T)*Sim['Z']
Sim['Stock $']=S_0*np.exp(Sim['X_T'])

#estimating VaR for simulated stock prices under portfolio A and D
Port=pd.DataFrame()
VaR=pd.DataFrame(index=['PA 95%', 'PA 97.5%', 'PD 95%','PD 97.5%'])
PA_0, PD_0 = 10000*S_0,10000*(S_0-0.26+3.65)
Port['Stock $'] = Sim['Stock $']
Port['PA_Loss'] = -10000*Sim['Stock $']+PA_0
Port['PD_Loss'] = Port['Stock $'].apply(lambda x: PDLoss(x))

#Port.sort_values(by=['PA_Loss'])['PA_Loss'].iloc[4751]
VaR['11/52 Day VaR Est.']=[Port.sort_values(by=['PA_Loss']).iloc[4751]['PA_Loss'],
              Port.sort_values(by=['PA_Loss']).iloc[4876]['PA_Loss'],
              Port.sort_values(by=['PD_Loss']).iloc[4751]['PD_Loss'],
              Port.sort_values(by=['PD_Loss']).iloc[4876]['PD_Loss'],
                     ]
"""How can I pass items from dataframe directly to query ?"""
VaR_PA_95=Port.sort_values(by=['PA_Loss']).iloc[4751]['PA_Loss']
VaR_PA_97=Port.sort_values(by=['PA_Loss']).iloc[4876]['PA_Loss']
VaR_PD_95=Port.sort_values(by=['PD_Loss']).iloc[4751]['PD_Loss']
VaR_PD_97=Port.sort_values(by=['PD_Loss']).iloc[4876]['PD_Loss']

VaR['11/52 Day ES Est.']=[
    Port.query("PA_Loss >= @VaR_PA_95")['PA_Loss'].mean(),
    Port.query("PA_Loss >= @VaR_PA_97")['PA_Loss'].mean(),
    Port.query("PD_Loss >= @VaR_PD_95")['PD_Loss'].mean(),
    Port.query("PD_Loss >= @VaR_PD_97")['PD_Loss'].mean()
]

#computing actual VaR from formula derived in 2c
VaR['11/52 Day VaR Act.'] = [674500*(1-np.exp(0.0052+0.091*norm.ppf(1-0.95))),
                            674500*(1-np.exp(0.0052+0.091*norm.ppf(1-0.975))),
                           8400,
                           8400]

# VaR.round(3)
Port

,Stock $,PA_Loss,PD_Loss
0,72.650202,-52002.023947,-18102.023947
1,74.036910,-65869.104218,-31969.104218
2,65.061130,23888.697067,8400.000000
3,72.173106,-47231.056963,-13331.056963
4,69.809506,-23595.063893,8400.000000
...,...,...,...
4995,62.798238,46517.615827,8400.000000
4996,63.978511,34714.888209,8400.000000
4997,73.856430,-64064.298567,-30164.298567
4998,67.773001,-3230.014873,8400.000000


#### 2e)

The VaR estimates are extremely close to their actual values calculated using the formula from 2c

#### 2f)

I prefer portfolio PD because it has a lower VaR for both confidence levels. 